In [81]:
import os
import ast
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

# 禁止随机，结果可复现
random.seed(42)
np.random.seed(42)

## 超参数选择

In [82]:
time_ratio = [1/2, 3/4, 7/8, 1][2]

In [83]:
# Parameters
emo_index_id = 0
stage_id = 0
data_number = 200

In [84]:
emo_index = ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'][emo_index_id]
stage = ['计算矩阵', '读取矩阵'][stage_id]
model = 'Deep-learning/BERT'

ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering'
Clustering_Method = 'DTW_KMeans'
Emotion_Data_PATH = f'{ROOT_PATH}/data/Emotion_Data/{model}'   # 情绪数据路径
Financial_Data_PATH = f'{ROOT_PATH}/data/Financial_Data' # 金融数据路径

print(f"Running with: emo_index={emo_index}, stage={stage}, model={model}, data_number={data_number}")

Running with: emo_index=上证综合情绪值, stage=计算矩阵, model=Deep-learning/BERT, data_number=200


## 数据准备

In [85]:
"""读取股吧个股的数据"""
all_data = []
file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

for file in file_list:
    file_path = os.path.join(Emotion_Data_PATH, file)
    df = pd.read_csv(file_path)
    stock_code = os.path.splitext(file)[0] # 获取股票编号（文件名去掉扩展名）
    
    # 提取每一行的日期和情绪值
    for _, row in df.iterrows():
        new_row = {
            '股票编号': stock_code,
            '日期': row['日期'],
            '上证综合情绪值': row['上证综合情绪值'],
            '沪深300情绪值': row['沪深300情绪值'],
            '创业板情绪值': row['创业板情绪值']
        }
        all_data.append(new_row)
        
guba_data = pd.DataFrame(all_data)
guba_data

,股票编号,日期,上证综合情绪值,沪深300情绪值,创业板情绪值
0,601933,2024-12-27,0.007,-0.044,-0.015
1,601933,2024-11-18,-0.155,-0.236,-0.354
2,601933,2024-11-17,0.001,0.022,0.004
3,601933,2024-11-17,-0.346,-0.337,-0.407
4,601933,2024-11-17,-0.247,-0.309,-0.358
...,...,...,...,...,...
495326,601919,2021-04-09,0.253,0.327,0.457
495327,601919,2021-04-09,-0.177,-0.190,-0.196
495328,601919,2021-04-08,0.218,0.194,0.167
495329,601919,2021-04-08,0.148,0.142,0.346


In [86]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2024-12-31


In [87]:
guba_data = guba_data.sort_values(by="日期").reset_index(drop=True)
idx_ratio = int(len(guba_data) * time_ratio)  
guba_data = guba_data.iloc[:idx_ratio]
guba_data

,股票编号,日期,上证综合情绪值,沪深300情绪值,创业板情绪值
0,601919,2021-01-01,-0.129,-0.132,-0.159
1,601012,2021-01-01,-0.483,-0.513,-0.641
2,601318,2021-01-01,-0.151,-0.183,-0.188
3,000625,2021-01-01,0.131,0.069,0.199
4,000725,2021-01-01,-0.129,-0.132,-0.159
...,...,...,...,...,...
433409,000063,2024-03-18,-0.079,-0.068,0.049
433410,600585,2024-03-18,-0.039,-0.062,-0.022
433411,000661,2024-03-18,0.006,0.022,0.075
433412,001979,2024-03-18,0.039,-0.020,0.028


In [88]:
# 查看最早和最晚日期
earliest_date = guba_data["日期"].min()
latest_date = guba_data["日期"].max()

print("最早日期：", earliest_date)
print("最晚日期：", latest_date)


最早日期： 2021-01-01
最晚日期： 2024-03-18


In [89]:
"""读取股票回报率的数据"""
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [90]:
# 进行左连接，guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = merged_data.dropna()
merged_data

,股票编号,日期,上证综合情绪值,沪深300情绪值,创业板情绪值,交易量,收益率变化
6439,601166,2021-06-01,-0.502,-0.540,-0.649,118168969.0,-0.022068
6440,002241,2021-06-01,-0.015,-0.039,0.031,119791643.0,0.034403
6441,601857,2021-06-01,0.081,0.104,0.160,128459929.0,0.019481
6442,300896,2021-06-01,0.080,0.086,0.235,2932326.0,0.004877
6443,300896,2021-06-01,-0.046,-0.017,-0.087,2932326.0,0.004877
...,...,...,...,...,...,...,...
433409,000063,2024-03-18,-0.079,-0.068,0.049,152776767.0,0.018621
433410,600585,2024-03-18,-0.039,-0.062,-0.022,15408992.0,-0.004683
433411,000661,2024-03-18,0.006,0.022,0.075,4668668.0,0.017536
433412,001979,2024-03-18,0.039,-0.020,0.028,68646843.0,0.012766


## 数据预处理

In [91]:
# Step 1: 对情绪值列进行 Min-Max 标准化
def min_max_normalization(df, cols):
    for col in cols:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df

# Step 2: 对每个股票编号的数据进行标准化和按日期汇总
def process_data(df):
    df_processed = pd.DataFrame()  # 用于存储结果
    for stock_code, stock_data in df.groupby('股票编号'):
        # 对每个股票编号内的数据进行标准化
        stock_data = min_max_normalization(stock_data, ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'])
        
        # 按日期汇总数据，同时保留股票编号
        stock_summary = stock_data.groupby('日期').agg({
            '股票编号': 'first',  # 保留股票编号（在同一日期内它是相同的，使用 'first'）
            '上证综合情绪值': 'mean',  # 上证综合情绪值按日期取均值
            '沪深300情绪值': 'mean',  # 沪深300情绪值按日期取均值
            '创业板情绪值': 'mean',  # 创业板情绪值按日期取均值
            '交易量': 'mean',  # 交易量按日期求和
            '收益率变化': 'mean'  # 收益率变化按日期取均值
        }).reset_index(drop=False)
        
        df_processed = pd.concat([df_processed, stock_summary], ignore_index=True)
    
    return df_processed

# 调用处理函数
final_data = process_data(merged_data)
final_data

,日期,股票编号,上证综合情绪值,沪深300情绪值,创业板情绪值,交易量,收益率变化
0,2021-06-01,000002,0.316338,0.267833,0.280789,60990961.0,-0.003745
1,2021-06-08,000002,0.347227,0.287914,0.300312,44676494.0,0.004626
2,2021-06-10,000002,0.325725,0.281754,0.289027,53800776.0,-0.010035
3,2021-06-11,000002,0.285945,0.224180,0.235237,75853738.0,-0.014035
4,2021-06-15,000002,0.314849,0.268572,0.283593,89915501.0,-0.020957
...,...,...,...,...,...,...,...
70090,2024-03-12,688981,0.353210,0.288720,0.212963,21003183.0,-0.002292
70091,2024-03-13,688981,0.322606,0.245732,0.161662,24307382.0,-0.000627
70092,2024-03-14,688981,0.281275,0.220122,0.116116,20417164.0,-0.024033
70093,2024-03-15,688981,0.326708,0.265854,0.199700,16195957.0,-0.000642


## DTW 聚类

In [92]:
# 选择需要的列
dtw_df = final_data[['日期', '股票编号', emo_index, '收益率变化']]

# 按照股票编号和日期排序
dtw_df = dtw_df.sort_values(by=['股票编号', '日期'])

# 创建一个以股票编号为键，日期为时间序列的字典
stock_data = {}
if data_number > len(dtw_df['股票编号'].unique()):
    data_number = len(dtw_df['股票编号'].unique())
for stock in dtw_df['股票编号'].unique()[:data_number]:
    stock_data[stock] = dtw_df[dtw_df['股票编号'] == stock][['日期', emo_index, '收益率变化']].reset_index(drop=True)

In [93]:
if stage == '计算矩阵':
    from tqdm import tqdm
    from fastdtw import fastdtw
    from scipy.spatial.distance import euclidean

    stock_ids = list(stock_data.keys())
    dtw_distances = np.zeros((len(stock_ids), len(stock_ids)))
    min_length = 3  # fastdtw 最小时间序列长度要求

    for i, stock_i in tqdm(enumerate(stock_ids), total=len(stock_ids), desc="计算 DTW 距离", unit="股票"):
        for j, stock_j in enumerate(stock_ids):
            if i < j:
                series_i = stock_data[stock_i][[emo_index, '收益率变化']].dropna().values
                series_j = stock_data[stock_j][[emo_index, '收益率变化']].dropna().values

                # 跳过长度太短的序列
                if len(series_i) < min_length or len(series_j) < min_length:
                    dtw_distances[i, j] = np.nan
                    dtw_distances[j, i] = np.nan
                    continue

                try:
                    distance, _ = fastdtw(series_i, series_j, dist=euclidean)
                    dtw_distances[i, j] = distance
                    dtw_distances[j, i] = distance  # 对称矩阵
                except Exception as e:
                    print(f"跳过 {stock_i} vs {stock_j}，错误: {e}")
                    dtw_distances[i, j] = np.nan
                    dtw_distances[j, i] = np.nan


计算 DTW 距离: 100%|██████████| 183/183 [13:09<00:00,  4.31s/股票]


In [94]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import os

# 用一个较大的值填充 NaN（也可以选择删除含 NaN 的行列）
filled_distances = np.where(np.isnan(dtw_distances), np.nanmax(dtw_distances) * 1.1, dtw_distances)

# KMeans 聚类
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(filled_distances)

# 创建一个 DataFrame 来保存股票编号与对应的聚类标签
stock_clusters_df = pd.DataFrame({
    '股票编号': stock_ids,
    '聚类标签': clusters
})

# 保存到 CSV
os.makedirs('output', exist_ok=True)
stock_clusters_df.to_csv(f'output/{time_ratio}.csv', index=False)

# 查看结果
stock_clusters_df


,股票编号,聚类标签
0,000002,1
1,000061,0
2,000063,0
3,000069,2
4,000100,2
...,...,...
178,603986,0
179,688005,0
180,688027,0
181,688029,0
